# 📌 Introduzione  

In questo notebook esploreremo come **ospitare ed eseguire modelli AI open-source con Ollama su Kaggle**, seguendo la guida proposta nell’articolo:  

🔗 [*Unleash the Power of AI: Host Your Own Ollama Models for Free with Google Colab*](https://medium.com/data-science-collective/unleash-the-power-of-ai-host-your-own-ollama-models-for-free-with-google-colab-0aac5f237a9f)  

## 🎯 Obiettivo  
L'obiettivo è testare la capacità di eseguire modelli **LLM (Large Language Models)** direttamente su **Kaggle**, sfruttando **Ollama**, una piattaforma progettata per semplificare il deployment di modelli AI su infrastrutture locali o cloud.  

## 🛠️ Cosa faremo?  
✅ Configureremo **Kaggle** per supportare **Ollama**  
✅ Scaricheremo e avvieremo **un modello AI open-source**  
✅ Effettueremo test di inferenza per valutarne le prestazioni  

Questa soluzione consente di **eseguire modelli AI gratuitamente** senza bisogno di infrastrutture complesse, rappresentando un'opzione interessante per sviluppatori e ricercatori che vogliono sperimentare con LLM senza costi aggiuntivi.  

## 📌 Prerequisiti  
🔹 Conoscenza di base di **Python** e dei **modelli AI** (opzionale, ma consigliata)  

📢 **Nota:**  
Poiché Kaggle ha alcune **limitazioni di tempo e risorse hardware**, valuteremo anche eventuali **ottimizzazioni** per migliorare l'esperienza d’uso.  


In [ ]:
!nproc

In [13]:
!apt-get install lshw

# Aggiornamento sistema e installazione CUDA
!apt-get install -y --no-install-recommends \
    cuda-toolkit-12-2 \
    libnccl2 \
    libnccl-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 129 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cuda-documentation-12-2 cuda-nsight-12-2 cuda-nsight-systems-12-2 cuda-nvvp-12-2 cuda-tools-12-2
  cuda-visual-tools-12-2 default-jre default-jre-headless gds-tools-12-2 libtinfo5 libxcb-icccm4
  libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0
  libxcb-xkb1 libxkbcommon-x11-0 libxtst6 nsight-systems-2023.2.3 openjdk-11-jdk-headless
  openjdk-11-jre openjdk-11-jre-headless
Suggested packages:
  openjdk-11-demo openjdk-11-source libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
Recom

In [1]:
# Impostazione variabili ambiente per multi-GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'  # Usa entrambe le GPU
os.environ['OLLAMA_NUM_GPU'] = '2'

# Ottimizzazioni NCCL per comunicazioni inter-GPU
os.environ['NCCL_ALGO'] = 'Ring'
os.environ['NCCL_NSOCKS_PERTHREAD'] = '4'
os.environ['NCCL_SOCKET_NTHREADS'] = '2'
os.environ['OLLAMA_GPU_MEMORY_PIN'] = '1'  # Previene swap memoria [1]
os.environ['OLLAMA_QUANTIZATION'] = 'q4_1'  # Balance prestazioni-memoria


In [2]:
# Installazione Python packages
!pip install -q nvidia-ml-py3 pyngrok ollama --quiet

  Preparing metadata (setup.py) ... done


In [3]:
# Download and install ollama to the system
!curl -fsSL https://ollama.com/install.sh | sh

# Verifica installazione
!ollama --version

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%###########################                                                             37.0%##########################                                                       43.3%###########################################                                                  48.7%###################################                                         58.8% 74.2%#############################################################################       95.1%########################      95.7%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import subprocess
import threading
import logging

# Configurazione logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def start_ollama_server():
    """Avvia il server Ollama con supporto multi-GPU"""
    cmd = "ollama serve"
    process = subprocess.Popen(
        cmd.split(),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )
    
    def log_output():
        """Thread per loggare l'output in tempo reale"""
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break
            if line:
                logger.info(line.strip())
    
    thread = threading.Thread(target=log_output, daemon=True)
    thread.start()
    
    print("🟢 Server Ollama avviato con 2 GPU")
    return process

server_process = start_ollama_server()

🟢 Server Ollama avviato con 2 GPU


In [5]:
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient

def setup_ngrok_tunnel(port=11434):
    """Crea un tunnel ngrok sicuro"""
    user_secrets = UserSecretsClient()
    ngrok_token = user_secrets.get_secret("NGROK_AUTHTOKEN")
    
    if not ngrok_token:
        raise ValueError("❌ Token Ngrok non trovato nei Kaggle Secrets!")
    
    ngrok.set_auth_token(ngrok_token)
    
    # Configurazione ottimizzata
    tunnel = ngrok.connect(
        port,
        host_header=f"localhost:{port}",
    )
    
    print(f"🔗 Tunnel Ngrok attivo: {tunnel.public_url}")
    return tunnel

In [6]:
NGROK_PORT = '11434'
ngrok_tunnel = setup_ngrok_tunnel(NGROK_PORT)

🔗 Tunnel Ngrok attivo: https://efc7-34-67-153-244.ngrok-free.app                                   


In [7]:
import time
import sys 

def ollama_run_model(model_name, timeout=300):
    """
    Esegue un modello Ollama con gestione degli errori
    
    Args:
        model_name (str): Nome del modello (es. 'gemma3:12b')
        timeout (int): Timeout in secondi
    """
    
    start_time = time.time()
    print(f"🚀 Avvio modello '{model_name}'...")
    
    try:
        cmd = f"ollama run {model_name}"
        with subprocess.Popen(
            cmd.split(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        ) as proc:
            
            # Live output handling
            while True:
                line = proc.stdout.readline()
                if not line and proc.poll() is not None:
                    break
                if line:
                    sys.stdout.write(line)
                    sys.stdout.flush()
                
                # Timeout check
                if time.time() - start_time > timeout:
                    raise TimeoutError(f"Timeout dopo {timeout} secondi")
            
            if proc.returncode != 0:
                raise RuntimeError(f"Errore nell'esecuzione (code: {proc.returncode})")
    
    except Exception as e:
        print(f"❌ Errore: {str(e)}")
        raise  
    finally:
        print(f"✅ Tempo esecuzione: {time.time()-start_time:.2f}s")

In [8]:
def ollama_remove_model(model_name, timeout=300):
    """
    Rimuove un modello Ollama con gestione degli errori
    
    Args:
        model_name (str): Nome del modello (es. 'gemma3:12b')
        timeout (int): Timeout in secondi
    """
    
    start_time = time.time()
    print(f"🚀 Rimozione modello '{model_name}'...")
    
    try:
        cmd = f"ollama rm {model_name}"
        with subprocess.Popen(
            cmd.split(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        ) as proc:
            
            # Live output handling
            while True:
                line = proc.stdout.readline()
                if not line and proc.poll() is not None:
                    break
                if line:
                    sys.stdout.write(line)
                    sys.stdout.flush()
                
                # Timeout check
                if time.time() - start_time > timeout:
                    raise TimeoutError(f"Timeout dopo {timeout} secondi")
            
            if proc.returncode != 0:
                raise RuntimeError(f"Errore nell'esecuzione (code: {proc.returncode})")
    
    except Exception as e:
        print(f"❌ Errore: {str(e)}")
        raise  
    finally:
        print(f"✅ Tempo esecuzione: {time.time()-start_time:.2f}s")

In [ ]:
# Esecuzione modelli
try:
    ollama_run_model("gemma3:4b")
    #ollama_run_model("gemma3:12b")
    #ollama_run_model("gemma3:27b")
    #ollama_run_model("deepseek-coder-v2:16b")
    #ollama_run_model("phi4:14b")
    #ollama_run_model("phi4-mini:3.8b")
    #ollama_run_model("llama3.3:70b")

    ##### DA PROVARE 
    # COGITO 3B-8B-14B 
    #ollama_run_model("cogito:8b") Context: 128K, Per abilitare ragionamento aggiungere al prompt: "Enable deep thinking subroutine"

    # DEEPCODER 14B
    # ollama_run_model("deepcoder:14b") Context: 64K (?)
except KeyboardInterrupt:
    print("Interruzione manuale rilevata")

In [11]:
# Rimozione modelli
try:
    ollama_remove_model("gemma3:4b")
    #ollama_remove_model("gemma3:12b")
    #ollama_remove_model("gemma3:27b")
    #ollama_remove_model("deepseek-coder-v2:16b")
    #ollama_remove_model("phi4:14b")
    #ollama_remove_model("phi4-mini:3.8b")
    #ollama_remove_model("llama3.3:70b")
except KeyboardInterrupt:
    print("Interruzione manuale rilevata")

🚀 Rimozione modello 'deepseek-coder-v2:16b'...
deleted 'deepseek-coder-v2:16b'
✅ Tempo esecuzione: 2.01s
